Assignment 2: Portfolio Optimization

In [ ]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import csv

## Get index of 4 tickers
tick4 = ["MSFT","GS","PG","SCHP"];

# Get variable names
with open('Prices.csv') as csvFile:
    reader = csv.reader(csvFile)
    tickers = next(reader) ## stores the tickers of all 390 stocks

tickind =[];
for t in tick4:
    tickind.append(tickers.index(t)) ## retrieve index that corresponds to each ticker

# Load data
prices = genfromtxt('Prices.csv', delimiter=',',skip_header = 1)

# get dimensions of data
d = prices.shape[0]
n = prices.shape[1]

# calculate monthly returns of each stock
returns = np.zeros((d-1,n))
for stock in range(n):
    for month in range(d-1):
        returns[month,stock] = prices[month+1,stock]/prices[month,stock]-1
        
# Store average return (parameter r_i in portfolio optimization model)       
avg_return = np.zeros(n)
avg_return = np.mean(returns,axis=0)

# Store covariance matrix (parameter C_ij in portfolio optimization model)
C = np.zeros((n,n))
C = np.cov(np.transpose(returns))

# Model 1

In [139]:
model = Model("PortfolioOptimization")

# Variable
w = {i: model.addVar(lb=0.0, ub=1.0, name=f"weight_{i}") for i in tickind}

In [140]:
# Constraint: The sum of weights must equal 1
model.addConstr(sum(w[i] for i in tickind) == 1, name="Weights_Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [141]:
# Constraint: Each individual weight must be greater than or equal to 0
for i in tickind:
    model.addConstr(w[i] >= 0, name=f"Non_negative_constraint_{i}")

In [142]:
# Constraint: The sum of weighted returns must be greater than 0.005
model.addConstr(sum(w[i] * avg_return[i] for i in tickind) >= 0.005, name="Sum_of_Weighted_Returns_Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [143]:
# Objective function: Minimize the portfolio variance
portfolio_variance = sum(w[i] * w[j] * C[i, j] for i in tickind for j in tickind)
model.setObjective(portfolio_variance, GRB.MINIMIZE)

In [144]:
# Solve the model
model.update()
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x3bac9f13
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.93770406e+03 -2.93770406e+03  4.

In [267]:
# Access the solution
if model.status == GRB.OPTIMAL:
    for i in tickind:
        print(f"{tickers[i]}: {round(w[i].x * 100, 3)}%")
    print(f"Total Portfolio Risk: {model.objVal}")

MSFT: 23.712%
GS: 2.586%
PG: 0.0%
SCHP: 73.702%
Total Portfolio Risk: 0.00017749363882836112


# Model 2

In [194]:
model_allstocks = Model("PortfolioOptimizationAcrossAllStocks")

# Collect indexes of all tickers in portfolio
tickind2 =[];
for t in tickers:
    tickind2.append(tickers.index(t))
    
w2 = {i: model_allstocks.addVar(lb=0.0, ub=1.0, name=f"weight_{i}") for i in tickind2}

In [195]:
# Constraint: The sum of weights must equal 1
model_allstocks.addConstr(sum(w2[i] for i in tickind2) == 1, name="Weights_Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [196]:
# Constraint: Each individual weight must be greater than or equal to 0
for i in tickind2:
    model_allstocks.addConstr(w2[i] >= 0, name=f"Non_negative_constraint_{i}")

In [197]:
# Constraint: The sum of weighted returns must equal 1
model_allstocks.addConstr(sum(w2[i] * avg_return[i] for i in tickind2) >= 0.005, name="Sum_of_Weighted_Returns_Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [198]:
# Objective function: Minimize the portfolio variance
portfolio_variance_allstocks = sum(w2[i] * w2[j] * C[i, j] for i in tickind2 for j in tickind2)
model_allstocks.setObjective(portfolio_variance_allstocks, GRB.MINIMIZE)

In [ ]:
# Solve the model
model_allstocks.update()
model_allstocks.optimize()

In [266]:
tolerance = 1e-6  # Adjust the tolerance level as needed

# Access the solution
if model_allstocks.status == GRB.OPTIMAL:
    for i in tickind2:
        if w2[i].x > tolerance:
            print(f"{tickers[i]}: {round(w2[i].x * 100, 3)}%")
    print(f"Total Portfolio Risk: {model_allstocks.objVal}")

ABBV: 1.135%
ABMD: 0.667%
ATVI: 1.369%
ANET: 1.175%
AIZ: 2.223%
ATO: 2.917%
BBY: 0.26%
CME: 1.275%
ED: 0.018%
DRI: 0.669%
RE: 0.235%
GWW: 0.777%
HAS: 0.246%
HCA: 2.955%
HUM: 2.267%
INFO: 5.173%
ICE: 1.418%
KEYS: 2.217%
LHX: 0.0%
LLY: 0.852%
LMT: 0.066%
PSX: 0.799%
PNC: 3.229%
BND: 68.056%
Total Portfolio Risk: 2.878503835027502e-05


# Model 3

In [297]:
model_atmost4stocks = Model("PortfolioOptimizationWithAtMost4Stocks")

w3 = {i: model_atmost4stocks.addVar(lb=0.0, ub=1.0, name=f"weight_{i}") for i in tickind2}

binary = {}
for i in tickind2:
    binary[i] = model_atmost4stocks.addVar(vtype = GRB.BINARY)

In [298]:
# Constraint: The sum of weights must equal 1
model_atmost4stocks.addConstr(sum(w3[i] for i in tickind2) == 1, name="Weights_Constraint")

# Constraint: Limit the number of stocks to at most 4
model_atmost4stocks.addConstr(sum(binary[i] for i in tickind2) <= 4, name="AtMost4Stocks_Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [299]:
# Constraint: Each individual weight must be greater than or equal to 0
for i in tickind2:
    model_atmost4stocks.addConstr(w3[i] >= 0, name=f"Non_negative_constraint_{i}")

In [300]:
# Constraint: The sum of weighted returns must equal 1
model_atmost4stocks.addConstr(sum(w3[i] * avg_return[i] for i in tickind2) >= 0.005, name="Sum_of_Weighted_Returns_Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [304]:
# Constraint: Weight of asset should be less than the present/absent value (whether present in portfolio or not)
for i in tickind2:
    model_atmost4stocks.addConstr(w3[i] <= binary[i])

In [302]:
# Objective function: Minimize the portfolio variance
portfolio_variance_atmost4stocks = sum(w3[i] * w3[j] * C[i, j] for i in tickind2 for j in tickind2)
model_atmost4stocks.setObjective(portfolio_variance_atmost4stocks, GRB.MINIMIZE)

In [303]:
# Solve the model
model_atmost4stocks.update()
model_atmost4stocks.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 783 rows, 780 columns and 2340 nonzeros
Model fingerprint: 0x6e451ca0
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.06s
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0009032

Root relaxation: objective 2.878501e-05, 129 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl

In [306]:
# Access the solution
if model_atmost4stocks.status == GRB.OPTIMAL:
    for i in tickind2:
        if binary[i].x == 1:
            print(f"{tickers[i]}: {round(w3[i].x * 100, 3)}%")
    print(f"Total Portfolio Variance: {model_atmost4stocks.objVal}")

CME: 12.641%
LLY: 7.548%
NVDA: 4.375%
BND: 75.436%
Total Portfolio Variance: 6.75347076072778e-05


## Question 1

**A. For Model 1, write down the optimal risk (i.e. the optimal objective function value), solver time, and the weight on each of the four stocks.**

In [269]:
print("Solver time for model 1 is %f" % model.Runtime, "seconds")
print("The weight of each of the four stocks is -")
if model.status == GRB.OPTIMAL:
    for i in tickind:
        print(f"{tickers[i]}: {round(w[i].x * 100, 3)}%")
print(f"Total Portfolio Variance: {round(model.objVal*100,4)}%")

Solver time for model 1 is 0.005153 seconds
The weight of each of the four stocks is -
MSFT: 23.712%
GS: 2.586%
PG: 0.0%
SCHP: 73.702%
Total Portfolio Variance: 0.0177%


**B. For Model 2, write down the optimal risk and solver time.**

In [268]:
print("Solver time for model 2 is %f" % model_allstocks.Runtime, "seconds")
print(f"Total Portfolio Variance: {round(model_allstocks.objVal*100,4)}%")

Solver time for model 1 is 0.004813 seconds
Total Portfolio Variance: 0.0029%


**C. For Model 3, report the optimal risk, solver time, and the ticker and weight on each of the
four stocks selected by the model**

In [313]:
print("Solver time for model 3 is %f" % model_atmost4stocks.Runtime, "seconds")
print("The weight of each of the four stocks is -")
if model_atmost4stocks.status == GRB.OPTIMAL:
    for i in tickind2:
        if binary[i].x == 1:
            print(f"{tickers[i]}: {round(w3[i].x * 100, 3)}%")
print(f"Total Portfolio Variance: {round(model_atmost4stocks.objVal*100,4)}%")

Solver time for model 3 is 3.010063 seconds
The weight of each of the four stocks is -
Total Portfolio Variance: 0.0068%


## Question 2
Use your solution to Question 1 above to answer the following questions:

**A. Is the optimal risk in Model 1 higher or lower than Model 2? Explain why in 1-2 sentences.**

* **Model 2 has lower optimal risk** - Model 2 has a risk of 0.0029% as comparedto Model 1, which has the risk of 0.0177%
* **Model 2 is more diversified** - With the increase in number of avaiable stocks from 4 to 390, the porfolio became more diversified. This diversification helps reduce risk as we can now choose stocks (more than 4) which are relatively less correlated and whose combination provides us with a sufficient return. This reduces the volatility in stocks, therefore having lower variance/risk

**B. Is the optimal risk in Model 2 higher or lower than Model 3? Explain why in 1-2 sentences.**

* **Model 2 has lower optimal risk** - Model 2 has a risk of 0.0029% as comparedto Model 1, which has the risk of 0.0068%
* **Model 3 is less diversified** - Model 3 can hold a portfolio of 1,2,3 or 4 stocks at best as compare to Model 2 which has 390 stocks at its disposal to create an optimal portfolio. The freedom to create a porfolio of more diverse assets (with a combination of more stocks that have lower correlation) is likely the reason for Model 2 outperforming Model 3 in risk. 

# Question 3

**A. Set Gurobi to terminate after 3 seconds by including XYZ.Params.TimeLimit = 3.0 in your code for Model 3, where ’XYZ’ is the name of your model. How does the objective function value at termination compare the optimal value obtained in question 1c)?**

In [318]:
model4 = Model("PortfolioOptimization - 4")

w4 = {i: model4.addVar(lb=0.0, ub=1.0, name=f"weight_{i}") for i in tickind2}

binary = {}
for i in tickind2:
    binary[i] = model4.addVar(vtype = GRB.BINARY)

# Constraint: The sum of weights must equal 1
model_atmost4stocks.addConstr(sum(w3[i] for i in tickind2) == 1, name="Weights_Constraint")

# Constraint: Limit the number of stocks to at most 4
model_atmost4stocks.addConstr(sum(binary[i] for i in tickind2) <= 4, name="AtMost4Stocks_Constraint")

# Constraint: Each individual weight must be greater than or equal to 0
for i in tickind2:
    model_atmost4stocks.addConstr(w3[i] >= 0, name=f"Non_negative_constraint_{i}")

# Constraint: The sum of weighted returns must equal 1
model_atmost4stocks.addConstr(sum(w3[i] * avg_return[i] for i in tickind2) >= 0.005, name="Sum_of_Weighted_Returns_Constraint")

# Constraint: Weight of asset should be less than the present/absent value (whether present in portfolio or not)
for i in tickind2:
    model_atmost4stocks.addConstr(w3[i] <= binary[i])

# Objective function: Minimize the portfolio variance
portfolio_variance_atmost4stocks = sum(w3[i] * w3[j] * C[i, j] for i in tickind2 for j in tickind2)
model_atmost4stocks.setObjective(portfolio_variance_atmost4stocks, GRB.MINIMIZE)

Set parameter TimeLimit to value 2.9


In [319]:
model4.update()
model4.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1173 rows, 780 columns and 3120 nonzeros
Model fingerprint: 0xe3500674
Model has 76245 quadratic objective terms
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 4e+00]
Presolved: 393 rows, 780 columns, 1950 nonzeros
Presolved model has 76245 quadratic objective terms

Continuing optimization...

 13740  2996     cutoff   36         0.00007    0.00005  20.9%  59.8   15s

Explored 17128 nodes (1028348 simplex iterations) in 2.91 seconds (2.90 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 6.75347e-05 

Time limit reached
Best objective 6.753470760728e-05, best bou

In [312]:
model_atmost4stocks.objVal

6.75347076072812e-05